# Hate Speech Binary Classification

Import all necessary libraries and install everything you need for training:

First, enable the GPU - under Accelerator on the right of the site, choose GPU. Be careful to always terminate the session (click the power off button), otherwise it will still be running and you will lose the 30 hours of GPU that you have available per week.

In [1]:
# install the libraries necessary for data wrangling, prediction and result analysis
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import torch
from numba import cuda
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
# Install transformers
# (this needs to be done on Kaggle each time you start the session)
!pip install -q transformers

In [3]:
# Install the simpletransformers
!pip install -q simpletransformers
from simpletransformers.classification import ClassificationModel

### Import the data
You might need to upload the data (click on the Add data button on the left of the site). I have uploaded the first version of the data that I created (see the 1-Data-Preparation.ipynb spreadsheet): "hatespeechdataset". If you change the Google Sheet, you can reprocess it by running the data preparation spreadsheet and upload the new version of it (go to the dataset description (https://www.kaggle.com/datasets/tajakuz/hatespeechdataset), click on the three dots and choose "New Version)

In [4]:
# Upload the binary hate speech dataset
hs_dataset = pd.read_csv("/kaggle/input/hatespeechdataset/hatespeech_binary_dataset.csv", sep="\t")
hs_dataset.head()

In [5]:
# See the statistics on the dataset
hs_dataset.describe()

In [6]:
# Define the labels
LABELS = [0,1]

In [7]:
# First, let's split the dataset into train and test split (70:30) by using the train_test_split from the sci-kit learn. We will shuffle the data beforehand and stratify it according to the labels (so that the distribution of labels is the same in both splits as in the original dataset)

hs_train, hs_test = train_test_split(hs_dataset, test_size=0.3, random_state=42, shuffle = True, stratify = hs_dataset.labels)

# See the size of the splits
hs_train.shape, hs_test.shape


In [8]:
# Check how the splits look like
hs_train.head(3)

In [9]:
# Check how the splits look like
hs_test.head(3)

In [10]:
# Create a file to save results into (you can find it under Data: Output). Be careful, run this step only once to not overwrite the results file.
results = []

with open("HateSpeech-Experiments-Results.json", "w") as results_file:
    json.dump(results,results_file, indent= "")

In [11]:
# In each next step (after the first experiment), open the results file instead of creating a new results file:
with open("./HateSpeech-Experiments-Results.json", "r") as results_file:
    previous_results = json.load(results_file)

# See the results
previous_results

## Training and testing - dummy classifier

Let's first apply a baseline classifier which predicts the most frequent class to each instance, to see what is the baseline score.

In [12]:
# Create X_train and Y_train parts, used for sci kit learning
# We need to split each split (test and train) into an object with just texts and object with just labels
X_train = list(hs_train.text)
Y_train = list(hs_train.labels)

X_test = list(hs_test.text)
Y_test = list(hs_test.labels)

# See their sizes
len(X_train), len(Y_train), len(X_test), len(Y_test)

In [13]:
# Use the Dummy Classifier, with the strategy "most_frequent"
dummy_clf = DummyClassifier(strategy="most_frequent")

# Train the model
dummy_clf.fit(X_train, Y_train)

#Get the predictions
y_pred = dummy_clf.predict(X_test)

In [14]:
# Compare the predictions with true values (Y_test)
micro = f1_score(Y_test, y_pred, labels=LABELS, average ="micro")
macro = f1_score(Y_test, y_pred, labels=LABELS, average ="macro")
accuracy = round(metrics.accuracy_score(Y_test, y_pred),3)
print(f"Micro F1: {micro:.3f}, Macro F1: {macro:.3f}, Accuracy: {accuracy}")

In [ ]:
# Save the results:
rezdict = {
    "model": "dummy",
    "microF1": micro,
    "macroF1": macro,
    "accuracy": accuracy,
    "y_true": Y_test.tolist(),
    "y_pred": {y_pred.tolist()}
    }
previous_results.append(rezdict)

In [16]:
previous_results

## Training and testing - Transformer model

We will use the basic English monolingual BERT model: https://huggingface.co/bert-base-uncased

You can find more documentation on how to use Simple Transformer models here: https://simpletransformers.ai/docs/usage/

For the hyperparameters (args), I used the ones that worked for me before, but you can see the entire list here: https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model

In [17]:
# Define the model
bertbase_model = ClassificationModel(
        "bert", "bert-base-cased",
        num_labels=2,
        use_cuda=True,
        args= {
    # Here, we have much more instances than in Experiment 1, so we can use less epochs.
    "num_train_epochs": 20,
    "labels_list": LABELS,
    "learning_rate": 1e-5,
    # We'll use a smaller max_seq_length (we could set it up to 512), because we have short texts
    "max_seq_length": 128,
    # Use this to mute the long output that tells you how the model proceeds.
    "silent": True,
    # Below are just some additional hyperparameters that we found that help with memory errors
    "save_steps": -1,
    "overwrite_output_dir": True,
    "no_cache": True,
    "no_save": True,
    }
    )

In [ ]:
# Train the model on train data - this will take some time
bertbase_model.train_model(hs_train)

print("Training is finished!")

In [ ]:
# Test the model - this will take some time

# Get the true labels
y_true = hs_test.labels

# Calculate the model's predictions on test
def make_prediction(input_string):
    return bertbase_model.predict([input_string])[0][0]

y_pred = hs_test.text.apply(make_prediction)

print("Testing is finished!")

In [ ]:
# Calculate the scores
macro = f1_score(y_true, y_pred, labels=LABELS, average="macro")
micro = f1_score(y_true, y_pred, labels=LABELS,  average="micro")
accuracy = round(metrics.accuracy_score(y_true, y_pred),3)
print(f"Macro f1: {macro:0.3}, Micro f1: {micro:0.3}, Accuracy: {accuracy}")

In [ ]:
# Save the results:
rezdict = {
    "model": "BERT",
    "epoch": 30,
    "microF1": micro,
    "macroF1": macro,
    "accuracy": accuracy,
    "y_true": Y_test.tolist(),
    "y_pred": y_pred.tolist(),
    }
previous_results.append(rezdict)

#Save intermediate results (just in case)
backup = []
backup.append(rezdict)
with open(f"backup-results.json", "w") as backup_file:
    json.dump(backup,backup_file, indent= "")

In [ ]:
# Compare the results by creating a dataframe from the previous_results dictionary:
results_df = pd.DataFrame(previous_results)

results_df

In [ ]:
print(results_df.drop(columns=["y_pred","epoch"]).to_markdown())

We can see that BERT performs better than the baseline and that we get the best results when we train the model for 20 epochs.

In [ ]:
# Add the end, save the file with results:
with open("./HateSpeech-Experiments-Results.json", "w") as final_results_file:
    json.dump(previous_results,final_results_file, indent= "")